#### 通过主成分分析进行无监督降维

关于特征选择和特征提取的区别：

* 特征选择：获取或者保留的是原始特征集合的一个子集，仍然是"original features"；
* 特征提取：是将待处理的数据集投影到一个新的特征空间。

特征提取可以理解为一种数据压缩方法，目的是保留大部分相关信息。在实践中，特征提取不仅用于改善存储空间或者学习算法的计算效率，还可以通过缓解维数灾难(curse of dimensionality)来提高预测性能，尤其是在使用正则化模型时。

#### 主成分分析(Principal component analysis, PCA)的主要步骤

无监督(unsupervised)的主成分分析方法, 帮助我们根据特征之间的相关性识别数据模式。简言之，PCA旨在找到高维数据中方差最大的方向，并将数据投影到与原始特征空间相同或更少的子空间上。

如下图所示，在新的特征轴相互正交的约束下，主成分可以理解为方差最大的方向：

In [3]:
from IPython.display import Image

Image(filename='../Python-MachineLearning/Attached_images/Chapter5/figures')

ModuleNotFoundError: No module named 'Ipython'